In [1]:
import pandas as pd
import datetime as datetime
import seaborn as sns
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
import os
import sys

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

sns.set()

In [2]:
data_pipeline_dir = os.path.abspath("../src/data_pipeline/")
sys.path.append(data_pipeline_dir)

from datapipeline import *

ModuleNotFoundError: No module named 'src'

In [ ]:
station_path = "../data/raw/dot_traffic_stations_2015.txt.gz"
traffic_df = "../data/raw/dot_traffic_2015.txt.gz"

In [ ]:
station_df = pd.read_csv(station_path, compression="gzip")

In [ ]:
traffic_df = pd.read_csv(traffic_df, compression="gzip")

# Data Cleaning

We take a glimpse of the data and clean both of the dataset before conducting any EDA. We will do this methodically by first cleaning the station dataset followed by the traffic dataset. 

## Station Data

In [ ]:
station_df.info()

In [ ]:
station_df.isnull().sum()

In [ ]:
station_df.head(10)

Based on the first 5 values of the dataset, we can already conclude that there will be missing values in some of the columns, how we deal with these missing values and columns will be based on further analysis.

One thing to also note is that there is around 55 columns of data in this particular station data.

In [ ]:
station_df.dtypes

I noticed that there are many columns that might be functionally the same and hence can be mapped to one another. For instance, direction_of_travel and direction_of_travel_name, will be creating some helper functions to help in obtaining mapping information.

In [ ]:
def print_mapped_value(df: pd.DataFrame, column1: str, column2: str) -> None:
    """
    Print mapped values from one column to another
    
    Args:
        df (DataFrame): DataFrame that contains colum1 and column2
        column1 (str): Column of the dataframe that can be mapped to column 2
        column2 (str): Column of the dataframe that can be mapped to column1
        
    Returns:
        None
    """
    column1_unique_values = df[column1].unique()
    for value in column1_unique_values:
        mapped_name = df.loc[
            station_df[column1] == value][column2].unique()

        if len(mapped_name) > 1:
            print(f"{value} maps to more than 1 value! they are {mapped_name}")
        else:
            print(f"{value} maps to {mapped_name}")

For this initial round of cleaning, we will be mainly focusing on remapping columns that are functionally the same and dropping one of them. We will then do EDA and try to make sense of missing values to see if there is any pattern to these missing values. Dropping any columns or rows will be our **last resort**.

### Algorithm of vehicle classification & Algorithm of vehicle classification name

In [ ]:
station_df["algorithm_of_vehicle_classification"].isnull().sum()

In [ ]:
station_df["algorithm_of_vehicle_classification_name"].isnull().sum()

Both of these columns have null value, although something seems a bit off. Intuitively if both of these columns are similar, the missing values of these two columns should be identical. We will have to delve into this relationship deeper.

In [ ]:
station_df["algorithm_of_vehicle_classification"].unique()

In [ ]:
station_df["algorithm_of_vehicle_classification_name"].unique()

In [ ]:
len(station_df["algorithm_of_vehicle_classification"].unique()) - len(station_df["algorithm_of_vehicle_classification_name"].unique())

Both the length of these two columns are different, which could explain why there is more missing values from one of the column

In [ ]:
len(station_df["algorithm_of_vehicle_classification"].unique())

In [ ]:
len(station_df["algorithm_of_vehicle_classification_name"].unique())

The algorithm_of_vehicle_classification column have two more unique values compared to the algorithm_of_vehicle_classification_name column

In [ ]:
unique_classification_value = station_df["algorithm_of_vehicle_classification"].unique()

In [ ]:
print_mapped_value(station_df, "algorithm_of_vehicle_classification", "algorithm_of_vehicle_classification_name")

We notice that the value 0, 1 maps directly to the nan value in the algorithm_of_vehicle_classification_name column. This recouncils the difference in unique value.

Since the algorithm_of_vehicle_classification column and the algorithm_of_vehicle_classification_name column are functionally the same, we will drop the algorithm_of_vehicle_classification_name column as it has more missing values and therefore less information.

In [ ]:
station_df.drop("algorithm_of_vehicle_classification_name", axis=1, inplace=True)

### Calibration of weighing system & Calibration of weighing system name

In [ ]:
station_df["calibration_of_weighing_system"].isnull().sum()

In [ ]:
station_df["calibration_of_weighing_system_name"].isnull().sum()

In [ ]:
len(station_df)

Both of these columns have null value, although something seems a bit off. similar to the algorithm_of_vehicle_column, if both of these columns are similar, the missing values of these two columns should be identical. We will have to delve into this relationship deeper.

The length of this dataset is only 28466. Having approximately 71% missing data might be too much missing information to work with. We might ultimately drop these two columns but let's explore this data further first.

In [ ]:
station_df["calibration_of_weighing_system"].unique()

In [ ]:
station_df["calibration_of_weighing_system_name"].unique()

In [ ]:
len(station_df["calibration_of_weighing_system_name"].unique()) - len(station_df["calibration_of_weighing_system"].unique())

The calibration_of_weighing_system column has 3 additional values compared to the calibration_of_weighing_system_name column which could explain why it has no null values. We would have to recouncil these difference and we can use the same process as the algorithm_of_weigh_classification column

In [ ]:
print_mapped_value(station_df, "calibration_of_weighing_system", "calibration_of_weighing_system_name")

The value 0, P and 2 maps to NaN in the calibration_of_weighing_system_name column, this reconcils the difference in unique values between the two columns and we will proceed to drop the calibration_of_weighing_system_name column as it has more null values. (Although we might still drop this column in the future as mentioned earlier).

Normally in situations like these, it will be important to ask the data provider for the cause of the difference but since it is not possible in this scenario, we will exercise some judgement and deal with the data in the most appropriate manner.

In [ ]:
station_df.drop("calibration_of_weighing_system_name", axis=1, inplace=True)

### Direction of Travel & Direction of Travel Name

In [ ]:
station_df["direction_of_travel"].isnull().sum()

In [ ]:
station_df["direction_of_travel_name"].isnull().sum()

Both of these columns have no null values, awesome!

In [ ]:
station_df["direction_of_travel"].unique()

In [ ]:
station_df["direction_of_travel_name"].unique()

In [ ]:
len(station_df["direction_of_travel"].unique()) - len(station_df["direction_of_travel_name"].unique())

In [ ]:
print_mapped_value(station_df, "direction_of_travel", "direction_of_travel_name")

In [ ]:
station_df.drop("direction_of_travel", inplace=True, axis=1)

### functional_classification & functional_classification_name

In [ ]:
len(station_df["functional_classification"].unique()) - len(station_df["functional_classification_name"].unique())

Both of these columns have the same number of unique values

In [ ]:
print_mapped_value(station_df, "functional_classification", "functional_classification_name")

This shows that there is a perfect mapping of the two columns and we should probably drop one of the columns to reduce dimensionality. We will be keeping functional_classification_name as it is more informative.

In [ ]:
station_df.drop("functional_classification", axis=1, inplace=True)

### lane_of_travel & lane_of_travel_name

In [ ]:
station_df["lane_of_travel"].isnull().sum()

In [ ]:
station_df["lane_of_travel_name"].isnull().sum()

In [ ]:
len(station_df["lane_of_travel"].unique()) - len(station_df["lane_of_travel_name"].unique())

Even though lane_of_travel and lane_of_travel_name have no null values, they have different cardinalities, this would mean that there might be a 1 to many mapping.

In [ ]:
print_mapped_value(station_df, "lane_of_travel", "lane_of_travel_name")

The relationship between lane_of_travel and lane_of_travel_name is one to many, to preserve as much information as possible, we will keep the lane_of_travel column over lane_of_travel_name.

In [ ]:
station_df.drop("lane_of_travel_name", axis=1, inplace=True)

### method_of_data_retrieval & method_of_data_retrieval_name

In [ ]:
station_df["method_of_data_retrieval"].isnull().sum()

In [ ]:
station_df["method_of_data_retrieval_name"].isnull().sum()

In [ ]:
len(station_df["method_of_data_retrieval"].unique()) - len(station_df["method_of_data_retrieval_name"].unique())

In [ ]:
print_mapped_value(station_df, "method_of_data_retrieval", "method_of_data_retrieval_name")

There is a perfect map, in the interest of verbosity, we will keep method_of_data_retrieval_name

In [ ]:
station_df.drop("method_of_data_retrieval_name", inplace=True, axis=1)

### method_of_traffic_volume_counting & method_of_traffic_volume_counting_name

In [ ]:
station_df["method_of_traffic_volume_counting"].isnull().sum()

In [ ]:
station_df["method_of_traffic_volume_counting_name"].isnull().sum()

In [ ]:
len(station_df["method_of_traffic_volume_counting"].unique()) - len(station_df["method_of_traffic_volume_counting_name"].unique())

In [ ]:
print_mapped_value(station_df, "method_of_traffic_volume_counting", "method_of_traffic_volume_counting_name")

We shall keep the method_of_traffic_volume_counting as it seems to have more cardinality and hence more information for our predictive model to work with

In [ ]:
station_df.drop("method_of_traffic_volume_counting_name", inplace=True, axis=1)

### method_of_truck_weighing & method_of_truck_weighing_name

In [ ]:
station_df["method_of_truck_weighing"].isnull().sum()

In [ ]:
station_df["method_of_truck_weighing_name"].isnull().sum()

In [ ]:
len(station_df["method_of_truck_weighing"].unique()) - len(station_df["method_of_truck_weighing_name"].unique())

In [ ]:
print_mapped_value(station_df, "method_of_truck_weighing", "method_of_truck_weighing_name")

It would be preferably to keep the name of the method as it will be more informative to do EDA without the actual name, however we would have to fill in the value nan with 0 as indicated from our mapping table. We will then drop the method_of_truck_weighing column

In [ ]:
station_df["method_of_truck_weighing_name"] = station_df["method_of_truck_weighing_name"].fillna("0")

In [ ]:
station_df.drop("method_of_truck_weighing", axis=1, inplace=True)

### method_of_vehicle_classification & method_of_vehicle_classification_name

In [ ]:
station_df["method_of_vehicle_classification"].isnull().sum()

In [ ]:
station_df["method_of_vehicle_classification_name"].isnull().sum()

In [ ]:
len(station_df["method_of_vehicle_classification"].unique()) - len(station_df["method_of_vehicle_classification_name"].unique())

In [ ]:
print_mapped_value(station_df, "method_of_vehicle_classification", "method_of_vehicle_classification_name")

Although we would prefer to keep the method_of_vehicle_classification_name, it seems like we will be losing information by doing so, hence we will be keeping mthe method_of_vehicle_classification instead.

In [ ]:
station_df.drop("method_of_vehicle_classification_name", inplace=True, axis=1)

### primary_purpose & primary_purpose_name

In [ ]:
station_df["primary_purpose"].isnull().sum()

In [ ]:
station_df["primary_purpose_name"].isnull().sum()

In [ ]:
len(station_df["primary_purpose"].unique()) - len(station_df["primary_purpose_name"].unique())

In [ ]:
print_mapped_value(station_df, "primary_purpose", "primary_purpose_name")

Since the primary_purpose column have more information than the primary_purpose_name column, we will keep that particular column instead.

In [ ]:
station_df.drop("primary_purpose_name", axis=1, inplace=True)

### sample_type_for_traffic_volume & sample_type_for_traffic_volume_name

In [ ]:
station_df["sample_type_for_traffic_volume"].isnull().sum()

In [ ]:
station_df["sample_type_for_traffic_volume_name"].isnull().sum()

In [ ]:
len(station_df["sample_type_for_traffic_volume"].unique()) - len(station_df["sample_type_for_traffic_volume_name"].unique())

In [ ]:
print_mapped_value(station_df, "sample_type_for_traffic_volume", "sample_type_for_traffic_volume_name")

Whether a station is used for traffic volume trends seems to be binary in nature, we can try mapping nans to the value 'Station not used for Traffic Volume Trends' while mapping the values "Y", "t" and "T" to 'Station used for Traffic Volume Trends', ie. 1

This is what will be done however one caveat is that we are assuming this column is binary in nature. With this in mind,we can just modify the sample_type_for_traffic_volume column and consolidate all under a uniform value of "T" and fillna to N

In [ ]:
station_df["sample_type_for_traffic_volume"].replace("N", "0", inplace=True)
station_df["sample_type_for_traffic_volume"].replace(["t", "Y", "T"], "1", inplace=True)
station_df["sample_type_for_traffic_volume"].fillna("0", inplace=True)

In [ ]:
station_df["sample_type_for_traffic_volume"] = pd.to_numeric(station_df["sample_type_for_traffic_volume"])

In [ ]:
station_df.drop("sample_type_for_traffic_volume_name", axis=1, inplace=True)

### sample_type_for_truck_weight & sample_type_for_truck_weight_name

In [ ]:
station_df["sample_type_for_truck_weight"].isnull().sum()

In [ ]:
station_df["sample_type_for_truck_weight_name"].isnull().sum()

In [ ]:
len(station_df["sample_type_for_truck_weight"].unique()) - len(station_df["sample_type_for_truck_weight_name"].unique())

In [ ]:
print_mapped_value(station_df, "sample_type_for_truck_weight", "sample_type_for_truck_weight_name")

With this mapping table in mind, we will keep the sample_type_for_truck_weigh as it retains the most possible information.

In [ ]:
station_df.drop("sample_type_for_truck_weight_name", axis=1, inplace=True)

### sample_type_for_vehicle_classification & sample_type_for_vehicle_classification_name

In [ ]:
station_df["sample_type_for_vehicle_classification"].isnull().sum()

In [ ]:
station_df["sample_type_for_vehicle_classification_name"].isnull().sum()

In [ ]:
len(station_df["sample_type_for_vehicle_classification"].unique()) - len(station_df["sample_type_for_vehicle_classification_name"].unique())

In [ ]:
print_mapped_value(station_df, "sample_type_for_vehicle_classification", "sample_type_for_vehicle_classification_name")

Similar to the sample_type_for_traffic_volume, we will assume that this column is binary in nature, ie. a station will only either be used for Heavy Vehicle Travel Information System or it will not be used.

We will map N and nans to 0 and Y, 2, T, H to 1.

In [ ]:
station_df["sample_type_for_vehicle_classification"].replace(["H", "Y", "2", "T"], "1",inplace=True)
station_df["sample_type_for_vehicle_classification"].replace("N", "0",inplace=True)
station_df["sample_type_for_vehicle_classification"].fillna("0", inplace=True)

In [ ]:
station_df["sample_type_for_vehicle_classification"] = pd.to_numeric(station_df["sample_type_for_vehicle_classification"])

In [ ]:
station_df.drop("sample_type_for_vehicle_classification_name", axis=1, inplace=True)

### type_of_sensor & type_of_sensor_name

In [ ]:
station_df["type_of_sensor"].isnull().sum()

In [ ]:
station_df["type_of_sensor_name"].isnull().sum()

In [ ]:
len(station_df["type_of_sensor"].unique()) - len(station_df["type_of_sensor_name"].unique())

In [ ]:
print_mapped_value(station_df, "type_of_sensor", "type_of_sensor_name")

Since there is perfect mapping, we can keep the more verbose column ie. type_of_sensor_name

In [ ]:
station_df.drop("type_of_sensor", inplace=True, axis=1)

We are now done with the station data (for now), moving on to the traffic data

## Traffic Data

In [ ]:
traffic_df.isnull().sum()

Most of the data in the traffic_df dataframe is filled up, only the restriction column seems to be missing a huge chunk of data. We will further investigate this particular column.

We will also dropped the functional_classification column and direction_of_travel column since we also dropped them in the station_df from our prior analysis

In [ ]:
traffic_df.drop(["functional_classification", "direction_of_travel"], axis=1, inplace=True)

### Restrictions

In [ ]:
traffic_df["restrictions"].unique()

The entire column are just NaN values, we will proceed to drop this column

In [ ]:
traffic_df.drop("restrictions", axis=1, inplace=True)

In [ ]:
# traffic_df.to_csv("../data/interim/dot_traffic_2015.csv", index=False)
# station_df.to_csv("../data/interim/dot_traffic_stations_2015.csv", index=False)

# Summary

These are the columns that were dropped from the station_df based on the analysis that we have done in the notebook:
1. algorithm_of_classification_name
2. calibration_of_weighing_system_name
3. direction_of_travel
4. functional_classification
5. lane_of_travel_name
6. method_of_data_retrieval_name
7. method_of_traffic_volume_counting_name
8. method_of_truck_weighing
9. method_of_vehicle_classification_name
10. sample_type_for_traffic_volume_name
11. sample_type_for_truck_weight_name
12. sample_type_for_vehicle_classification_name
13. type_of_sensor

These are the columns that were dropped frm the traffic_df based on the analysis that was done:

1. restrictions

We will move on to the EDA portion, data cleaning is by no means finished, we might change the way we clean the data based on the EDA that we will be doing.

# EDA

In [ ]:
# station_path = "../data/interim/dot_traffic_stations_2015.csv"
# traffic_path = "../data/interim/dot_traffic_2015.csv"

In [ ]:
# traffic_df = pd.read_csv(traffic_path)
# station_df = pd.read_csv(station_path)

Creating some helper functions to clean the data and for feature engineering.

In [ ]:
def create_max_volume_column(df: pd.DataFrame) -> pd.DataFrame:
    """
    Create a daily max volume column for EDA purpose by summing up all the traffic_volume columns
    
    Args:
        df (pd.DataFrame): Dataframe to modify
        
    Returns:
        modified_df (pd.DataFrame): DataFrame with new total_volume column
    """
    volume_columns = [column for column in df.columns if column.startswith("traffic_volume")]
    df["total_volume"] = df[volume_columns].sum(axis=1)
    
    return df


def convert_established_year_to_int(df: pd.DataFrame) -> pd.DataFrame:
    """
    Convert the year_station_established to int by adding a 2000 to any number between 0 and 15, while adding a
    1900 to any number between 16 and 99. Assumption is based on the dataset being from 2015 ie. it is not possible to have 2016
    
    Args:
        df (pd.DataFrame): Dataframe to modify
        
    Returns:
        modified_df (pd.DataFrame): DataFrame with modified established_year column
    """
    df["year_station_established"] = df["year_station_established"].apply(lambda x: 2000 + x if x <= 15 else 1900 + x)
    
    return df

def create_years_of_operation_column(df: pd.DataFrame) -> pd.DataFrame:
    """
    Create a year of operation column that computes the number of years that the observation station has been in service
    
    Args:
        df (pd.DataFrame): Dataframe to create the year of operation column
        
    Returns:
        new_df (pd.DataFrame): DataFrame with a new year_of_operation column
    """
    df["year_of_data"] = df["year_of_data"] + 2000
    df["year_of_service"] = df["year_of_data"] - df["year_station_established"]
    
    return df

Since we will be doing traffic volume prediction for New York specifically, we can drop the irrelevant stations that are not situated in New York. Based on https://www.nrcs.usda.gov/wps/portal/nrcs/detail/?cid=nrcs143_013696, the state code for New York is 36

Lets do some EDA on the respective df to see if we can generate any informative insights from them. Since we are trying to predict traffic volume for the New York's rush hour

In [ ]:
NEW_YORK_CODE = 36

In [ ]:
station_df = station_df[station_df["fips_state_code"] == NEW_YORK_CODE]
traffic_df = traffic_df[traffic_df["fips_state_code"] == NEW_YORK_CODE]

Since all the fips_state_code will be of only one value, we can safely drop this column also

In [ ]:
station_df.drop("fips_state_code", inplace=True, axis=1)
traffic_df.drop("fips_state_code", inplace=True, axis=1)

After dropping selectively dropping the irrelevant stations, let's combine the two dataframe into one combined dataframe for the EDA

In [ ]:
combined_df = traffic_df.merge(station_df,
                               on=["station_id", "direction_of_travel_name",
                                   "functional_classification_name", "lane_of_travel",
                                   "year_of_data"])

In [ ]:
combined_df.describe()

Based on a simple describe function, we can already tell there's some columns that should not be used such as method_of_data_retrieval, method_of_traffic_volume_counting as they only have unique value which will be relatively useless in helping our prediction. Will be noted down and removed later

It will be nice to have a total_volume for a particular day for EDA on a daily level

In [ ]:
combined_df = create_max_volume_column(combined_df)

Lets check the common columns to make sure they agree with each other as a sanity check

Based on the basic info here, we will address missing values, and also what columns to keep for modelling before we move on to feature engineering and preprocessing the data to be used for modelling.

Based on the information above, we will definitely drop the shrp_site_identification column as it has 0 (!) non-null values and there's literally nothing for us to work with.

We will also do some sanity check to ensure we merged correctly ie. the number of rows for traffic_df should be exactly the same as combined_df.

In [ ]:
len(combined_df) == len(traffic_df)

### year_of_station_established

For easier sorting of the data, we will change the year_station_established column to int and also convert them to its full form. ie. 98 -> 1998, 0 -> 2001. We will assume that any number greater 15 is from the 1900s and any number between 0 and 15 is from the 2000s since the dataset is from 2015

In [ ]:
combined_df = convert_established_year_to_int(combined_df)
station_df = convert_established_year_to_int(station_df)

In [ ]:
plot_dims = (15, 11)
fig, ax= plt.subplots(figsize=plot_dims)
plt.title("Count of Year Station is Established")
sns.countplot(station_df["year_station_established"])
plt.show()

Most of the observation stations are relatively new where they are constructed some time during the 2000s. However we have an exceeding number of stations that are built during the 1960s also. 

Since we will be working in time series, it will be useful to convert the date column to a datetime dtype in pandas

In [ ]:
combined_df["date"] = pd.to_datetime(combined_df["date"])

In [ ]:
plot_dims = (15, 11)
fig, ax= plt.subplots(figsize=plot_dims)
plt.title("Average volume per year of station is established")
sns.barplot(x=combined_df.groupby("year_station_established").mean().index, y="total_volume", 
        data=combined_df.groupby("year_station_established").mean())
plt.show()

Not much relationship can be derived from this chart although we can note that the older stations are probably kept as they are typically in high volume places which can be seen that their total volume seem to be slightly above average.

### Day of Week Seasonality Trend

In [ ]:
day_groupby = combined_df.groupby("day_of_week").sum()["total_volume"]

In [ ]:
plot_dims = (15, 11)
fig, ax= plt.subplots(figsize=plot_dims)
plt.title("Volume by Day")
sns.barplot(x=day_groupby.index, y=day_groupby, ax=ax)
plt.show()

We will assume for this dataset that day_7 is Sunday and day_1 is Monday, so on and so forth.

Based on the traffic volume, we can see that other than Monday, all the other week days have the highest volume of traffic. This could be due to people commuting to work.

### Month Seasonality Trend

In [ ]:
month_groupby = combined_df.groupby("month_of_data").sum()["total_volume"]

In [ ]:
plot_dims = (15, 11)
fig, ax= plt.subplots(figsize=plot_dims)
plt.title("Volume by Month")
sns.barplot(x=month_groupby.index, y=month_groupby, ax=ax)
plt.show()

The volume across months seems to be fairly consistent however there seem to be a giant spike in traffic volume during the month of December. This could be due to Americans typically travelling across the country to visit their relatives during the festive seasons such as Christmas.

### Hourly Seasonality Trend

In [ ]:
volume_columns = [column for column in combined_df.columns if column.startswith("traffic_volume")]
hourly_groupby = combined_df[volume_columns].sum()

In [ ]:
# rename column for clarity
for column in volume_columns:
    name = column.split("_")[4]
    hourly_groupby = hourly_groupby.rename(index={column: name})
    

In [ ]:
plot_dims = (15, 11)
fig, ax= plt.subplots(figsize=plot_dims)
plt.title("Volume by Hour of the Day")
sns.barplot(x=hourly_groupby.index, y=hourly_groupby, ax=ax)
plt.show()

Not surprisingly, odd hours such as 10pm to 5am have the lowest traffic volume among all the hours of the day. Volume starts to spike at around 6am as people start commuting to their workplace and delivery fleets start their day. Traffic volume increase continuously throughout the day until around 5pm to 6pm when it starts declining as people start to knock off and commute home where they rest and stop travelling on roads.

Based on Wikipedia, rush hour is typically between 6am to 10am (morning) and 3pm to 7pm (evening) which seems to agree with this plot where traffic volume is typically at its highest.

### Lane of Travel

Lane of travel is in int dtype when it is actually a mapping, converting it back to category for visualisation purposes

In [ ]:
combined_df["lane_of_travel"] = combined_df["lane_of_travel"].astype("object")

In [ ]:
lane_of_travel_groupby = combined_df.groupby("lane_of_travel").sum()
lane_of_travel_groupby.reset_index(inplace=True)

In [ ]:
plot_dims = (15, 11)
fig, ax= plt.subplots(figsize=plot_dims)
plt.title("Volume by lane")
sns.barplot(y="total_volume", x="lane_of_travel", data=lane_of_travel_groupby)
plt.show()

Based on the prior mapping, lane_of_travel value of 1 is the right (outer most lane) while lane 2 to 5 are "Other Lanes".

Intuitively, lane 1 would definitely have the most volume of traffic since all roads would have a minimum of at least 1 lane wheras having 2 to 5 lanes is definitely not a given. Hence this data does make sense as to why the volume of traffic is in chronological order.

### Direction of Travel

In [ ]:
direction_of_travel_groupby = combined_df.groupby("direction_of_travel_name").sum()
direction_of_travel_groupby.reset_index(inplace=True)

In [ ]:
plot_dims = (12, 10)
fig, ax= plt.subplots(figsize=plot_dims)
plt.title("Total Volume by Travel")
sns.barplot(x="direction_of_travel_name", y="total_volume", data=direction_of_travel_groupby)
plt.show()

Total traffic volume for the direction of travel is the greatest for the South, which makes sense since New York is situated in Northern US and it most people would be more likely to travel south towards Central America 

### Sensor Type

In [ ]:
combined_df["type_of_sensor_name"].unique()

We mainly have 4 types of sensor in this dataset, they are:

1. Inducatance Loop
2. Sonic/Acoustic
3. Microwave
4. Piezoelectric

we can take a look at the average volume for each of the sensors

In [ ]:
combined_df = create_years_of_operation_column(combined_df)
sensor_groupby = combined_df.groupby("type_of_sensor_name").mean().reset_index()

In [ ]:
plot_dims = (12, 10)
fig, ax= plt.subplots(figsize=plot_dims)
plt.title("Average Total Volume by Sensor")
sns.barplot(x="type_of_sensor_name", y="total_volume", data=sensor_groupby)
plt.show()

In [ ]:
plot_dims = (12, 10)
fig, ax= plt.subplots(figsize=plot_dims)
plt.title("Average Service Year by Sensor")
sns.barplot(x="type_of_sensor_name", y="year_of_service", data=sensor_groupby)
plt.show()

In [ ]:
plot_dims = (12, 10)
fig, ax= plt.subplots(figsize=plot_dims)
plt.title("Count of Sensor Type")
sns.countplot(station_df["type_of_sensor_name"])
plt.show()

In [ ]:
combined_df.groupby("type_of_sensor_name").min()["year_of_service"]

In [ ]:
combined_df.groupby("type_of_sensor_name").max()["year_of_service"]

Based on our analysis above, there are a few interesting takeaways.

1. Piezoelectric has the most average total volume among all the sensors even though inductance loop is the most commonly used sensor type. 

2. Inducatance loop is also the sensor that is the oldest sensor type and also the most commonly used method (inducatance loop). It is still used today and was even still used 55 years ago!

### Longitude and Latitude

We will be working with station_df data instead since we only need the station specific information. However, based on my external research, I've noticed some discrepency in the longitude data. Finding longitude between 72.20 and 79.75 brought me to India (Using the current latitude).

After experimenting, it seems like the longitude data should be negative. Will be cleaning up the data in this manner.

In [ ]:
station_df["longitude"].min()

In [ ]:
station_df["longitude"].max()

In [ ]:
station_df["latitude"].min()

In [ ]:
station_df["latitude"].max()

In [ ]:
station_df["longitude"] = station_df["longitude"] * -1
combined_df["longitude"] = station_df["longitude"] * -1

By obtaining the minimum value of both longitude and latitude, we can plot out where each of the station is located in New York City by overlaying a bounding box (similar to object detection in Computer Vision) where we have a matrice of ([min_longitude, max_longitude], [min_latitude, max_latitude]) to encapsulate every single station in our dataset before plotting the stations to ensure we capture all the points.

In [ ]:
maps = plt.imread("map.png")

In [ ]:
bbox = ((station_df["longitude"].min(), station_df["longitude"].max(),
         station_df["latitude"].min(), station_df["latitude"].max()))

In [ ]:
fig, ax = plt.subplots(figsize = (30, 40))
ax.scatter(station_df["longitude"], station_df["latitude"])

ax.imshow(maps, extent=bbox)

After converting longitude to negative, it seems that we are in the vicinity of New York, however some of the observation stations reside in the water (see Lake Ontario). There is a possibility that we did not manipulate the data correctly or that there might be some issues during data collection.

### hpms_sample_identifier & hpms_sample_type

In [ ]:
combined_df.loc[(combined_df["hpms_sample_identifier"].isnull()) & (combined_df["hpms_sample_type"] != "N")]

Based on this we can map that the null values for hpms_sample_identifier directly maps to hpms_sample_type. We can choose to keep one of them.

In [ ]:
hpms_groupby = combined_df.groupby("hpms_sample_identifier").mean()

In [ ]:
plot_dims = (12, 10)
fig, ax= plt.subplots(figsize=plot_dims)
plt.title("HPMS Sample Identifier")
sns.barplot(x=hpms_groupby.index, y="total_volume", data=hpms_groupby)
plt.show()

Seems like the specific hpms_sample_identifier has an impact on the total volume of the traffic, hence we will keep the hpms_sample_identifier column over the hpms_sample_type_column

In [ ]:
combined_df.drop("hpms_sample_type", axis=1, inplace=True)

In [ ]:
combined_df.info()

### algorithm_of_vehicle_classification

In [ ]:
combined_df["algorithm_of_vehicle_classification"].isnull().sum()

In [ ]:
combined_df["algorithm_of_vehicle_classification"].unique()

Since the algorithm of vehicle classification column only has F or NaN, we will drop this column as there's two possibility. Either the entire columns have "F" as its value ie. does not contribute to prediction, or it could be an unknown algorithm which we do not know. As it comprises of approximate 40% of the data, we will drop the data to be safe

## Feature Engineering

Some interesting features we can engineer from this dataset could be the total traffic volume in a day. This could be useful when we are trying to determine if there's any seasonality based on months.

We can also engineer a new feature where we determine the number of years that the station has been in operation. We can do this by subtracting the year 2015 with the year where the station is established. We should probably do a sanity check that the year_station_discontinued is 0 and the year_of_data is all 15

### years_of_operation

In [ ]:
combined_df["year_station_discontinued"].unique()

All stations should be operational, hence this data makes sense and we can just drop this since it does not offer any new information.

In [ ]:
combined_df["year_of_data"].unique()

The data only comprises of 2015 data which is why the year of data column only have 15 inside.

In [ ]:
combined_df = create_years_of_operation_column(combined_df)

Created a new column indicating the number of years that the observation station is being used and then dropping the year of data and year_station_discontinued column as we don't need it anymore (one unique value only).

In [ ]:
combined_df.info()

## Preprocessing

Final round of cleaning where we remove any columns with only one unique value as it will not help the model in predicting

In [ ]:
for column in combined_df.columns:
    if len(combined_df[column].unique()) == 1:
        combined_df.drop(column, inplace=True, axis=1)

The station_id itself will also not be helpful in predicting traffic volume and we will remove it. We will also remove any locational information as from the previous EDA, it seems like the location data might not be entirely accurate and that might be discrepency in the data collection process (will not be possible to determine the cause of problem without communicating with the data collector).

We will also drop the date column as we have already dissected that particular column into other columns such as day_of_data, day_of_week which will capture all the information provided by the date column

In [ ]:
combined_df.drop(["station_id", "fips_county_code", "longitude", "latitude",
                  "previous_station_id", "day_of_data", "day_of_week", "month_of_data"], inplace=True, axis=1)

As we will be predicting the evening rush hour for New York City, we will have to drop columns after 7pm to prevent data leakage as in a real world scenario we will not have information after 8pm when predicting traffic volume for that particular day.

In [ ]:
time_column_to_drop = []

for column in combined_df.columns:
    if column.startswith("traffic_volume"):
        hour = column.split("_")[-1]
        if int(hour) > 1900:
            time_column_to_drop.append(column)

for column in time_column_to_drop:
    combined_df.drop(column, inplace=True, axis=1)
        

In [ ]:
combined_df.info()

## Summary

Based on the EDA, some of the interesting insights that we can derive are:

1. 3pm to 7pm has the heaviest traffic along with 6am to 10pm (Evening and morning rush hour respectively)
2. December have a spike of traffic (could be travelling out of New York to visit their family for the holiday seasons)
3. Weekdays typically have heavier traffic compared to weekend (People could be spending more time at home instead of commuting to work.
4. Most observation stations are relatively new (built after 2000s) although there is an anomaly in that there is a lot of stations built durign the 1960s (could be any stations that are built before 1960 are automatically binned in 1960)

and these are some of the features that was engineered:

1. year_of_operation
2. max_volume

and these are the columns that were dropped:
1. method_of_data_retrieval
2. method_of_traffic_volume_counting
3. year_of_data
4. year_station_discontinued
5. year_station_established
6. record_type
7. hpms_sample_type
8. station_id
9. longitude & latitude
10. fips_county_code
11. previous_station_id
12. day_of_data
13. day_of_week
14. month_of_data
15. algorithm_of_vehicle_classification

The next step is to prepare the data to be fed into the model, this is where will have to do one hot encoding for categorical data and scaling for numerical columns before they can be used for any form of model-related activities.